<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/RotinaDeClassificacaoDaBaseDeDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Base de Dados Twitter

Análise da base de dados do Twitter com linguagem ofensiva

Classificação

Import e instalação das bibliotecas necessárias

In [2]:
#Install e downloads
!pip install emoji
!pip install pandas
!pip install nltk
#nltk.download('all-nltk') #Demora um pouco

     |████████████████████████████████| 174 kB 5.0 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=886a89fb63bb6410726f954cbf686cbea04905ea4aa4faf8da21e3fe3d35b6ee
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [3]:
#Imports
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import re
import itertools

In [41]:
########## PATHS DE ARQUIVOS ##########

# Base após pré-processamento COM stopwords (Base 1)
PATH_BASE1_COM_STOPWORDS = r"/content/drive/MyDrive/TCC/dados/processadas/Base1_Com_StopWords.csv"  

# Base após pré-processamento SEM stopwords (Base 2)
PATH_BASE2_SEM_STOPWORDS = r"/content/drive/MyDrive/TCC/dados/processadas/Base2_Sem_StopWords.csv"

# Amostra Base 1
PATH_TESTE_BASE1 = r"/content/drive/MyDrive/TCC/dados/processadas/Teste_Base1_Com_StopWords.csv"  

# Amostra Base 2
PATH_TESTE_BASE2 = r"/content/drive/MyDrive/TCC/dados/processadas/Teste_Base2_Sem_StopWords.csv"

# Lista de Hates
PATH_LIST_HATE_EXPRESSIONS_PT_BR = r"/content/drive/MyDrive/TCC/dados/dicionarios/odio_list.csv"

In [37]:
#Carrega a base de dados com stopWords
#base1 = pd.read_csv(PATH_BASE1_COM_STOPWORDS, index_col=0)
base1 = pd.read_csv(PATH_TESTE_BASE1, index_col=0)
base1.head()

,id_str,created_at,text,entities.hashtags,place.full_name,text_limpo
0,1037067688953167879,Tue Sep 04 19:59:39 +0000 2018,"O famoso: Deus me livre, mas quem me dera... h...",[],"Rio de Janeiro, Brazil",o famoso deus me livre mas quem me dera
1,1101497244325502977,Fri Mar 01 14:59:42 +0000 2019,Eu to ficando maluco ou o Weverton do Palmeira...,[],"Divinópolis, Brasil",eu estou ficando maluco ou o weverton do palme...
2,1085552210191233025,Wed Jan 16 14:59:50 +0000 2019,👉🏾 Hoje: SAMBINHA DE QUARTA \r\n📍Hashtag Bar -...,[],"Rio de Janeiro, Brazil",hoje sambinha de quarta hashtag bar seabra fil...
3,1151869182373482501,Thu Jul 18 14:59:48 +0000 2019,@rick_pcf @ygorcac 2017 de primeira achei feio...,[],"Fortaleza, Brazil",2017 de primeira achei feio talvez por ser mei...
5,1074695634513469440,Mon Dec 17 15:59:40 +0000 2018,P= mt chato mas eu gosto mt❤️,[],"Maricá, Brasil",p muito chato mas eu gosto muito


In [42]:
#Carrega a base de dados sem stopWords
#base2= pd.read_csv(PATH_BASE2_SEM_STOPWORDS, index_col=0)
base2= pd.read_csv(PATH_TESTE_BASE2, index_col=0)
base2.head()

,id_str,created_at,text,entities.hashtags,place.full_name,text_limpo,text_cleanStopWords
0,1037067688953167879,Tue Sep 04 19:59:39 +0000 2018,"O famoso: Deus me livre, mas quem me dera... h...",[],"Rio de Janeiro, Brazil",o famoso deus me livre mas quem me dera,famoso deus livre dera
1,1101497244325502977,Fri Mar 01 14:59:42 +0000 2019,Eu to ficando maluco ou o Weverton do Palmeira...,[],"Divinópolis, Brasil",eu estou ficando maluco ou o weverton do palme...,ficando maluco weverton palmeiras convocado se...
2,1085552210191233025,Wed Jan 16 14:59:50 +0000 2019,👉🏾 Hoje: SAMBINHA DE QUARTA \r\n📍Hashtag Bar -...,[],"Rio de Janeiro, Brazil",hoje sambinha de quarta hashtag bar seabra fil...,sambinha quarta hashtag bar seabra filho sexta...
3,1151869182373482501,Thu Jul 18 14:59:48 +0000 2019,@rick_pcf @ygorcac 2017 de primeira achei feio...,[],"Fortaleza, Brazil",2017 de primeira achei feio talvez por ser mei...,2017 achei feio talvez meio diferente demorei ...
5,1074695634513469440,Mon Dec 17 15:59:40 +0000 2018,P= mt chato mas eu gosto mt❤️,[],"Maricá, Brasil",p muito chato mas eu gosto muito,chato gosto


In [43]:
base1.shape

(773, 6)

In [44]:
base2.shape

(773, 7)

Classificação Hate, Hate+ e outros

In [10]:
# Carrega dicionário hate
dict_hate = pd.read_csv(PATH_LIST_HATE_EXPRESSIONS_PT_BR)
dict_hate.head(5)

,hatelist
0,aberração
1,abestado
2,abestalhado
3,aborto
4,acabouamamata


In [33]:
# Definição da função de classificação de hate e not_hate

def classifica_hate(tweet):
  # Labels de retorno
  HATE = "hate"
  NOT_HATE = "not_hate"
  
  for palavra_hate in dict_hate['hatelist']:
    result = re.subn(r'^' + palavra_hate + '\s', palavra_hate + ' ', tweet) # Substitui no início
    if(result[1]):
      break
    result = re.subn(r'\s' + palavra_hate + '$',' '+ palavra_hate, tweet) # Substitui no final
    if(result[1]):
      break
    result = re.subn(r'\s' + palavra_hate + '\s', ' '+palavra_hate+ ' ', tweet) #Substitui no meio
    if(result[1]):
      break

  if(result[1]):
    return HATE
  else:
    return NOT_HATE

In [45]:
# Executa a classificação na base 1
base1['classificacao'] = base1['text_limpo'].apply(lambda x: classifica_hate(x))
base1.head()

,id_str,created_at,text,entities.hashtags,place.full_name,text_limpo,classificacao
0,1037067688953167879,Tue Sep 04 19:59:39 +0000 2018,"O famoso: Deus me livre, mas quem me dera... h...",[],"Rio de Janeiro, Brazil",o famoso deus me livre mas quem me dera,not_hate
1,1101497244325502977,Fri Mar 01 14:59:42 +0000 2019,Eu to ficando maluco ou o Weverton do Palmeira...,[],"Divinópolis, Brasil",eu estou ficando maluco ou o weverton do palme...,not_hate
2,1085552210191233025,Wed Jan 16 14:59:50 +0000 2019,👉🏾 Hoje: SAMBINHA DE QUARTA \r\n📍Hashtag Bar -...,[],"Rio de Janeiro, Brazil",hoje sambinha de quarta hashtag bar seabra fil...,not_hate
3,1151869182373482501,Thu Jul 18 14:59:48 +0000 2019,@rick_pcf @ygorcac 2017 de primeira achei feio...,[],"Fortaleza, Brazil",2017 de primeira achei feio talvez por ser mei...,hate
5,1074695634513469440,Mon Dec 17 15:59:40 +0000 2018,P= mt chato mas eu gosto mt❤️,[],"Maricá, Brasil",p muito chato mas eu gosto muito,not_hate


In [46]:
# Salva a base1
base1.to_csv("base1_classificada.csv")